## Часть 2. Тематическое моделирование 
    Постройте несколько тематических моделей коллекции документов с разным числом тем. Приведите примеры понятных (интерпретируемых) тем.
    Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). Можете ли вы их прокомментировать / объяснить?



In [1]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from googletrans import Translator
from scipy.spatial.distance import cosine
from tqdm import tqdm
import compress_fasttext
%matplotlib inline
%pylab inline



Populating the interactive namespace from numpy and matplotlib


In [2]:


responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [03:15, 1027.43it/s]


In [3]:
train_list = []
for resp in responses:
#     print(list(resp.values()))
    train_list.append(list(resp.values()))
train = pd.DataFrame(columns = list(responses[99].keys()), data = train_list)
del train_list

In [4]:
train.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


In [6]:
train[train['rating_grade'] == 5]

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
98,г. Санкт-Петербург,False,"Хорошо, когда проблемы решаются",0,лицензия № 1073,apka45,Русславбанк,2015-06-03 17:40:38,Досрочно закрывали вклады перед отпуском. Жена...,5.0
122,г. Нижний Новгород,False,Изменяют проценты по уже открытому договору вк...,3,лицензия № 226,20157673,РосинтерБанк,2015-06-04 09:38:18,"Хотел открыть вклад ""Доходный+"" под 19% год , ...",5.0
157,г. Москва,False,"Отделение Марксистская, дом 5: спасибо за помощь!",0,лицензия № 1623,ospar,ВТБ 24,2015-06-04 22:58:38,"Вчера, 3 июня, просидев в очереди более 3-х ча...",5.0
259,г. Москва,False,Хорошее обслуживание,0,лицензия № 3206,ser-gyr,Соверен Банк,2015-06-02 18:23:41,Сегодня открывал вклад в этом банке. Очередь б...,5.0
266,г. Москва,False,Моя история получения кредита в Ренессанс Кред...,1,лицензия № 3354,SkaIa,Ренессанс Кредит,2015-06-03 02:03:28,Моя история получения кредита в Ренессанс Кред...,5.0
...,...,...,...,...,...,...,...,...,...,...
153488,None,False,Повысилось качество обслуживания,0,лицензия № 1623,Атмос,ВТБ 24,2005-04-13 12:00:00,Пользуюсь банком 3 месяца - специально открыл ...,5.0
153489,None,False,Стабильный банк с высокими процентами,0,лицензия № 1971,Александра,Ханты-Мансийский банк Открытие,2005-04-14 13:41:00,А что писать? Ставки по вкладам высокие (до 13...,5.0
153491,None,False,Ну тогда копите денюжки и покупайте не в креди...,4,лицензия № 2289,irzik,Русский Стандарт,2005-04-14 17:42:00,А я с мужем обставила техникой совершенно нову...,5.0
153494,None,False,Не все так страшно,0,лицензия № 2557,Вера,Ситибанк,2005-05-18 14:38:00,"Слишком большой банк, не всегда учитывает нашу...",5.0


In [8]:
%%time
import re
from nltk.corpus import stopwords
regex = re.compile("[А-Яа-я]+")

mystopwords = stopwords.words('russian') 
from pymystem3 import Mystem

m = Mystem()

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "


train.text = train.text.apply(lemmatize)
train.text = train.text.str.lower()
train.text = train.text.apply(words_only)
train.text = train.text.apply(remove_stopwords)  




CPU times: user 6min 25s, sys: 16.3 s, total: 6min 41s
Wall time: 57min


In [10]:
from gensim.corpora import *
texts = [train.text.iloc[i].split() for i in range(len(train))]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [17]:
%%time
from gensim.models import  *
from gensim import similarities

tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# index = similarities.MatrixSimilarity(corpus_tfidf)
# sims = index[corpus_tfidf]

CPU times: user 4.94 s, sys: 284 ms, total: 5.23 s
Wall time: 5.29 s


## Постройте несколько тематических моделей коллекции документов с разным числом тем. Приведите примеры понятных (интерпретируемых) тем.


In [23]:
%%time
lsi = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=15)

CPU times: user 3min 53s, sys: 11.1 s, total: 4min 4s
Wall time: 3min 29s


In [24]:
lsi.show_topics(15)

[(0,
  '0.193*"карта" + 0.132*"кредит" + 0.115*"деньги" + 0.108*"счет" + 0.108*"заявление" + 0.101*"банкомат" + 0.100*"вклад" + 0.100*"отделение" + 0.095*"сумма" + 0.095*"сбербанк"'),
 (1,
  '-0.441*"вклад" + 0.338*"кредит" + -0.176*"банкомат" + 0.157*"платеж" + 0.153*"страховка" + 0.148*"задолженность" + -0.142*"карта" + -0.142*"открывать" + 0.137*"погашение" + -0.119*"очередь"'),
 (2,
  '-0.448*"вклад" + 0.298*"банкомат" + 0.237*"карта" + 0.193*"сбербанк" + -0.173*"кредит" + 0.134*"операция" + -0.133*"договор" + 0.131*"деньги" + 0.127*"заявление" + 0.125*"средство"'),
 (3,
  '-0.271*"вклад" + -0.218*"сумма" + -0.192*"руб" + 0.191*"номер" + 0.181*"заявка" + -0.165*"платеж" + -0.149*"процент" + -0.148*"погашение" + -0.147*"рубль" + -0.146*"договор"'),
 (4,
  '-0.433*"карта" + 0.227*"банкомат" + 0.206*"номер" + -0.191*"лимит" + 0.184*"вклад" + -0.182*"кредитный" + 0.154*"звонок" + 0.124*"терминал" + -0.123*"страховка" + 0.109*"ваш"'),
 (5,
  '0.403*"банкомат" + -0.277*"вклад" + 0.192*"с

In [20]:
%%time
lsi_5 = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=5)

CPU times: user 3min 30s, sys: 8.97 s, total: 3min 39s
Wall time: 3min 12s


In [21]:
lsi_5.show_topics(5)

[(0,
  '0.193*"карта" + 0.133*"кредит" + 0.115*"деньги" + 0.108*"счет" + 0.108*"заявление" + 0.100*"вклад" + 0.100*"банкомат" + 0.099*"отделение" + 0.096*"сбербанк" + 0.096*"сумма"'),
 (1,
  '-0.446*"вклад" + 0.338*"кредит" + 0.163*"страховка" + 0.159*"платеж" + 0.147*"задолженность" + -0.145*"открывать" + -0.141*"карта" + 0.140*"погашение" + -0.136*"банкомат" + -0.126*"очередь"'),
 (2,
  '-0.394*"вклад" + 0.260*"банкомат" + 0.246*"карта" + -0.198*"кредит" + 0.198*"сбербанк" + 0.141*"операция" + -0.141*"страховка" + 0.135*"средство" + 0.127*"деньги" + 0.124*"заявление"'),
 (3,
  '-0.271*"вклад" + 0.233*"номер" + -0.207*"сумма" + 0.181*"звонок" + 0.181*"заявка" + -0.176*"руб" + -0.156*"договор" + -0.151*"платеж" + -0.147*"процент" + -0.144*"погашение"'),
 (4,
  '0.331*"вклад" + 0.253*"номер" + 0.218*"звонок" + -0.205*"страховка" + -0.193*"заявка" + -0.160*"банкомат" + 0.156*"ваш" + 0.132*"персональный" + 0.127*"удалять" + 0.124*"база"')]

### По тематическому моделированию с 15 темами можно обьяснить следующие  выделенные темы:


(0,
  '0.193*"карта" + 0.133*"кредит" + 0.115*"деньги" + 0.108*"счет" + 0.108*"заявление" + 0.100*"вклад" + 0.100*"банкомат" + 0.099*"отделение" + 0.096*"сбербанк" + 0.096*"сумма"')
  **сбербанк - открытие вклада/карты/кредита**
  
 (1,
  '-0.446*"вклад" + 0.338*"кредит" + 0.163*"страховка" + 0.159*"платеж" + 0.147*"задолженность" + -0.145*"открывать" + -0.141*"карта" + 0.140*"погашение" + -0.136*"банкомат" + -0.126*"очередь"')
  **открытие  и погоашение вклада/кредита**
 
 (2,
  '-0.394*"вклад" + 0.260*"банкомат" + 0.246*"карта" + -0.198*"кредит" + 0.198*"сбербанк" + 0.141*"операция" + -0.141*"страховка" + 0.135*"средство" + 0.127*"деньги" + 0.124*"заявление"')
  **операции**

 (3,
  '-0.271*"вклад" + 0.233*"номер" + -0.207*"сумма" + 0.181*"звонок" + 0.181*"заявка" + -0.176*"руб" + -0.156*"договор" + -0.151*"платеж" + -0.147*"процент" + -0.144*"погашение"')
  **Заявка на вклад/кредит. Условия**

(4,
  '0.331*"вклад" + 0.253*"номер" + 0.218*"звонок" + -0.205*"страховка" + -0.193*"заявка" + -0.160*"банкомат" + 0.156*"ваш" + 0.132*"персональный" + 0.127*"удалять" + 0.124*"база"')
 **Оформление вклада/кредиат по персональному предложению**


In [25]:
%%time
lda = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=15,
                        alpha='auto', eta='auto', iterations = 20, passes = 5)


CPU times: user 36min 49s, sys: 1min 48s, total: 38min 37s
Wall time: 21min 33s


In [26]:

lda.show_topics(15)

[(0,
  '0.031*"это" + 0.014*"весь" + 0.013*"говорить" + 0.012*"мочь" + 0.011*"сказать" + 0.011*"деньги" + 0.010*"просто" + 0.009*"свой" + 0.009*"знать" + 0.008*"девушка"'),
 (1,
  '0.055*"кредит" + 0.038*"банк" + 0.035*"платеж" + 0.032*"сумма" + 0.020*"задолженность" + 0.018*"кредитный" + 0.017*"месяц" + 0.017*"погашение" + 0.016*"год" + 0.015*"оплачивать"'),
 (2,
  '0.035*"день" + 0.029*"банк" + 0.021*"позвонить" + 0.017*"ответ" + 0.015*"сказать" + 0.015*"звонить" + 0.014*"вопрос" + 0.012*"получать" + 0.012*"неделя" + 0.012*"отделение"'),
 (3,
  '0.164*"счет" + 0.058*"деньги" + 0.045*"перевод" + 0.040*"банк" + 0.022*"платеж" + 0.022*"комиссия" + 0.018*"открывать" + 0.017*"переводить" + 0.015*"средство" + 0.011*"выписка"'),
 (4,
  '0.047*"телефон" + 0.039*"номер" + 0.033*"банк" + 0.031*"звонить" + 0.030*"звонок" + 0.016*"альфа" + 0.013*"человек" + 0.013*"трубка" + 0.012*"который" + 0.011*"это"'),
 (5,
  '0.114*"вклад" + 0.038*"банк" + 0.036*"открывать" + 0.031*"процент" + 0.026*"депози

In [27]:
%%time
lda = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5,
                        alpha='auto', eta='auto', iterations = 20, passes = 5)


CPU times: user 31min 4s, sys: 1min 33s, total: 32min 38s
Wall time: 17min 21s


In [28]:

lda.show_topics(5)

[(0,
  '0.066*"карта" + 0.027*"банк" + 0.025*"счет" + 0.021*"деньги" + 0.020*"банкомат" + 0.009*"это" + 0.009*"операция" + 0.008*"сумма" + 0.007*"снимать" + 0.007*"рубль"'),
 (1,
  '0.018*"день" + 0.017*"сказать" + 0.017*"банк" + 0.015*"это" + 0.012*"деньги" + 0.012*"отделение" + 0.012*"звонить" + 0.011*"позвонить" + 0.011*"мочь" + 0.010*"говорить"'),
 (2,
  '0.043*"банк" + 0.014*"сотрудник" + 0.011*"ваш" + 0.011*"клиент" + 0.010*"который" + 0.010*"данный" + 0.009*"г" + 0.008*"телефон" + 0.008*"ответ" + 0.008*"свой"'),
 (3,
  '0.028*"банк" + 0.013*"это" + 0.013*"клиент" + 0.010*"вклад" + 0.009*"весь" + 0.009*"очень" + 0.008*"отделение" + 0.008*"очередь" + 0.008*"работать" + 0.007*"свой"'),
 (4,
  '0.060*"кредит" + 0.038*"банк" + 0.022*"сумма" + 0.020*"кредитный" + 0.018*"договор" + 0.017*"платеж" + 0.013*"год" + 0.010*"месяц" + 0.010*"процент" + 0.010*"документ"')]

In [29]:
from IPython.display import Audio

Audio(url="https://wav-sounds.com/wp-content/uploads/2017/10/Herbert-06.wav",autoplay=True)

## Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). Можете ли вы их прокомментировать / объяснить?

    Среди тем чаще всего встречается упоминание сберабнка (реже - альфабанка). Выделение этих банков в ключевые слова может быть связано с общиом количеством клиентов и большой долей рынка, которые эти банки занимают

